In [2]:
import pandas as pd 
import altair as alt 
import os 
from glob import glob

In [94]:
# all_df = []
# for file in glob("results/*demux.txt"):
#     demux_df = pd.read_csv(f"{file}", sep=",")
#     # First columns contains filename.... I missed the name 
#     demux_df = demux_df.rename({demux_df.columns[0]: "filename"}, axis=1)
    
#     demux_df["well"] = demux_df[demux_df.columns[0]].str.split(".", expand=True)[0]
#     #Create pool name 
#     demux_df["pool"] = os.path.basename(file).replace(".demux.results","")
    
#     # Join with barcode info 
#     demux_df = demux_df.set_index("well")

#     barcode_file = file.replace(".demux.txt", ".barcode.txt")
#     barcode_df = pd.read_csv(barcode_file, sep="\t", header=None).set_index(0)
#     barcode_df.columns = ["barcode","statut", "comment"]
#     demux_df = demux_df.join(barcode_df)    
    
#     # append to all 
#     all_df.append(demux_df)
    
# all_df = pd.concat(all_df).reset_index()
# all_df.to_csv("all_results.txt") 

all_df = pd.read_csv("all_results.txt")

In [95]:
alt.Chart(all_df[all_df.pool.str.contains(r"190")]).mark_point().encode(
    x="pN1",
    y="pN2",
    color="statut",
    tooltip="comment").properties(width=800, height=300)

alt.Chart(...)

In [92]:
alt.Chart(all_df[all_df.pool.str.contains(r"191")]).mark_point().encode(
    x="pN1",
    y="pN2",
    color="statut",
    tooltip="comment").properties(width=800, height=300)

alt.Chart(...)

In [93]:
alt.Chart(all_df[all_df.pool.str.contains(r"192")]).mark_point().encode(
    x="pN1",
    y="pN2",
    color="statut",
    tooltip="comment").properties(width=800, height=300)

alt.Chart(...)